In [ ]:
# This pipeline is to be run in Vertex Workbench
# Install Kubeflow Pipelines and GCP AI Platform
!pip3 install kfp --user -q
!pip3 install --upgrade google-cloud-platform --user -q

In [ ]:
from datetime import date, datetime
from typing import NamedTuple # for passing data between steps
import google.cloud.aiplatform as aip
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component

In [ ]:
PROJECT_ID = ""
BUCKET_NAME = ""
BUCKET_URI = f"gs://{BUCKET_NAME}"
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline_root/control"
TIMESTAMP = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

In [ ]:
# Pre-processing component
# To do: integrate Andre's component here
@component(
    packages_to_install=[
        "pandas",
        "google-cloud-storage",
        "google-cloud-aiplatform"
    ])
def create_automl_import_file_op(bucket_name: str) -> NamedTuple('Outputs', [('uri', str), ('size', int)]):
  # Todo: integrate Andre's work
  return ("gs://test", 1000)

In [ ]:
@dsl.pipeline(
    name="iowa-steam-sentiment-pipeline",
    pipeline_root=PIPELINE_ROOT
)

def pipeline():
  import_op = create_automl_import_file_op(BUCKET_NAME)

  ds_op = gcc_aip.TextDatasetCreateOp(
      project=PROJECT_ID,
      display_name=f"iowa-steam-reviews-processed-{TIMESTAMP}",
      gcs_source=import_op.outputs['uri'],
      import_schema_uri='aiplatform.schema.dataset.ioformat.text.single_label_classification',
      sync=sync)
  ds_op.wait()

  # todo: chain together training of model